In [22]:
import pandas as pd
import numpy as np

In [3]:
#preprocessing

lnames = ['movie_id','imdb_id','tmdb_id']
links = pd.read_csv('ml-latest-small/links.csv', sep=',', header=None, names=lnames)

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('ml-latest-small/ratings.csv', sep=',', header=None, names=rnames)

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('ml-latest-small/movies.csv', sep=',', header=None, names=mnames)

In [14]:
data = pd.merge(pd.merge(movies, ratings), links)
data = data.drop([0])#first row is labels
data[:5]

,movie_id,title,genres,user_id,rating,timestamp,imdb_id,tmdb_id
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,0114709,862
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,0114709,862
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,0114709,862
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,0114709,862
5,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,0114709,862


In [16]:
#Each movie average rating
data['rating'] = pd.to_numeric(data['rating'], downcast="float")
data['timestamp'] = data['timestamp'].astype(int)
mean_ratings = data.pivot_table(values='rating', index='title', aggfunc='mean')
mean_ratings

,rating
title,
'71 (2014),4.000000
'Hellboy': The Seeds of Creation (2004),4.000000
'Round Midnight (1986),3.500000
'Salem's Lot (2004),5.000000
'Til There Was You (1997),4.000000
...,...
eXistenZ (1999),3.863636
xXx (2002),2.770833
xXx: State of the Union (2005),2.000000


In [17]:
#Highest rated movies
top_movies = mean_ratings.sort_values(['rating'], ascending=False)
top_movies[:10]

,rating
title,
Gena the Crocodile (1969),5.0
True Stories (1986),5.0
Cosmic Scrat-tastrophe (2015),5.0
Love and Pigeons (1985),5.0
Red Sorghum (Hong gao liang) (1987),5.0
"Thin Line Between Love and Hate, A (1996)",5.0
Lesson Faust (1994),5.0
Eva (2011),5.0
Who Killed Chea Vichea? (2010),5.0


In [18]:
#removing outliers 
ratings_by_title = data.groupby('title').size()
ratings_by_title

title
'71 (2014)                                    1
'Hellboy': The Seeds of Creation (2004)       1
'Round Midnight (1986)                        2
'Salem's Lot (2004)                           1
'Til There Was You (1997)                     2
                                             ..
eXistenZ (1999)                              22
xXx (2002)                                   24
xXx: State of the Union (2005)                5
¡Three Amigos! (1986)                        26
À nous la liberté (Freedom for Us) (1931)     1
Length: 9719, dtype: int64

In [19]:
#titles with atleast 1% of the number of ratings as the most rated movie
min_rating = max(ratings_by_title)*0.01
min_rating = int(min_rating)
print(min_rating)
active_titles = ratings_by_title.index[ratings_by_title >= 10]
active_titles

3


Index([''burbs, The (1989)', '(500) Days of Summer (2009)',
       '10 Cloverfield Lane (2016)', '10 Things I Hate About You (1999)',
       '10,000 BC (2008)', '101 Dalmatians (1996)',
       '101 Dalmatians (One Hundred and One Dalmatians) (1961)',
       '12 Angry Men (1957)', '12 Years a Slave (2013)', '127 Hours (2010)',
       ...
       'Zack and Miri Make a Porno (2008)', 'Zero Dark Thirty (2012)',
       'Zero Effect (1998)', 'Zodiac (2007)', 'Zombieland (2009)',
       'Zoolander (2001)', 'Zootopia (2016)', 'eXistenZ (1999)', 'xXx (2002)',
       '¡Three Amigos! (1986)'],
      dtype='object', name='title', length=2269)

In [60]:
#Highest rated movies with enough ratings to qualify
mean_ratings = mean_ratings.loc[active_titles]
mean_ratings = mean_ratings.sort_values(by='rating', ascending=False)
mean_ratings

,rating
title,
Secrets & Lies (1996),4.590909
Guess Who's Coming to Dinner (1967),4.545455
Paths of Glory (1957),4.541667
"Streetcar Named Desire, A (1951)",4.475000
"Celebration, The (Festen) (1998)",4.458333
...,...
Battlefield Earth (2000),1.657895
"Flintstones in Viva Rock Vegas, The (2000)",1.625000
Speed 2: Cruise Control (1997),1.605263


In [65]:
data['user_id'] = data['user_id'].astype(str).astype(int)
sorted_by_user = data.sort_values(by = 'user_id')
sorted_by_user.tail(100)

,movie_id,title,genres,user_id,rating,timestamp,imdb_id,tmdb_id
37375,1967,Labyrinth (1986),Adventure|Fantasy|Musical,610,5.0,1493879438,0091369,13597
99891,158872,Sausage Party (2016),Animation|Comedy,610,3.5,1493848024,1700841,223702
96554,107771,Only Lovers Left Alive (2013),Drama|Horror|Romance,610,5.0,1479544448,1714915,152603
82257,44974,Hard Candy (2005),Drama|Thriller,610,3.5,1493849640,0424136,2652
93132,88125,Harry Potter and the Deathly Hallows: Part 2 (...,Action|Adventure|Drama|Fantasy|Mystery|IMAX,610,4.5,1493846733,1201607,12445
...,...,...,...,...,...,...,...,...
71057,6387,Once a Thief (Zong heng si hai) (1991),Action|Comedy|Crime|Thriller,610,3.5,1479542038,0101020,47423
87455,59915,Stuck (2007),Horror|Thriller,610,3.5,1493848708,0758786,13848
97037,111663,Zombeavers (2014),Action|Comedy|Horror,610,3.5,1493848822,2784512,254474
98182,122886,Star Wars: Episode VII - The Force Awakens (2015),Action|Adventure|Fantasy|Sci-Fi|IMAX,610,4.5,1479544589,2488496,140607


In [66]:
#Data Encoding: each user and their movie ratings
n_users = data.user_id.unique().shape[0]
print(n_users, ' users')
n_movies = data.movie_id.unique().shape[0]
print(n_movies, ' movies')
ratings = []
sorted_by_user = data.sort_values(by = 'user_id')
sorted_by_user[:5]
for row in sorted_by_user.itertuples():
    #print(row)
    ratings.append((int(row[4]),int(row[1]),float(row[5])))
ratings = pd.DataFrame(ratings, columns=['user', 'movie', 'rating'])    
ratings

610  users
9724  movies


,user,movie,rating
0,1,1,4.0
1,1,1777,4.0
2,1,1732,5.0
3,1,1676,3.0
4,1,50,5.0
...,...,...,...
100831,610,6387,3.5
100832,610,59915,3.5
100833,610,111663,3.5
100834,610,122886,4.5
